### 1D-Finite-Elemente Beispiel. Implizite Lösung der Wärmeleitungsgleichung mit der Crank-Nicolson Methode

Dieser Code basiert auf einem Beispiel von Claudio Belli. Der Code wurde etwas modifiziert, da er für diese Anwendungen nicht funktionierte. Es stellte sich heraus, dass ein Vektor für die Randbedingungen fehlte (unten als Vektor a bezeichnet) und die Formulierung der Randbedingungen nicht hinreichend war.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os, sys
import matplotlib

matplotlib.rc('font', size=18)
matplotlib.rc('font', family='Arial')

Hier die Parameter für die Aufgabe aus Philpotts.

In [2]:
k = 1E-6       # m2/s Thermische Diffusivität für Gesteine liegt ungefähr in dieser Größenordnung
Tr=25.         # Temperatur an der Oberfläche des Lavasees (konstant, Randbedingung)
Tl=1200.       # Temp am Boden des Lavasees (konstant, Randbedingung)
Ti=1200.       # initiale T der Masse unterhalb der Oberfläche
dt = 1800      # seconds
tsteps = 12    # number of time steps
nplot = 1      # number of timesteps before plotting

#definition of numerical parameters
N = 17 #51 #number of grid points  z, die Koeff.Matrizen A und B haben den Rang N-2
#L = float(1) #size of grid  z
dz = 0.0424

L = dz*(N-1)
#dx = L/(N-1) #grid spacing

r = k*dt/dz**2 
print(r, L)

1.00124599501602 0.6784


Hier die Parameter einer anderen Aufgabe, die zu Kontrollzwecken gerechnet wurde. Es handelt sich um einen Stab der Länge 5 cm und der initialen Temperatur von 20°C, an dessen einem Ende eine Temperatur von 100°C und an dessen anderem Ende eine Temperatur von 25°C angelegt wird. In Schritten von 1 cm wird die Temperaturverteilung in den Stab nach 3, 6 und 9 Sekunden berechnet.

In [3]:
r=0.4239
L=0.05
dt=3
dz=0.01
tsteps=3
N=6
Tr=25.
Tl=100.
Ti=20.

Jetzt der Code für die eigentlichen Berechnungen. Es werden zunächst die Matrizen definiert. Danach folgt die Berechnung des Vektors u, der die Temperaturen für die verschiedenen Tiefen (z) bei dem jeweils nächsten Zeitschritt j+1 enthält. Die verschiedenen print-Anweisungen dienen dem Degugging und können auskommentiert werden. Im unteren Teil wird ein Diagramm erstellt. Dieses muss für andere Berechnungen ggf. angepasst werden.

In [4]:
#initialize matrices A, B and a and b arrays.
# A und B sind die Koeffizientenmatrizen. Es sind beides Diagonalmatrizen.
# Man benötigt N-2 Elemente bzw. Bestimmungsgleichungen, da es 2 Randbedingungen gibt.
# Die Vektoren a and b definieren die Randbedingungen (in diesen Fällen die Temperaturen rechts und links)
# Die vollständige Gleichung lautet A u + a = B u + b  | mit i+1 auf der linken Seite und i auf der rechten
# Ziel ist die Berechnung des Vektors u (für i+1, d.h. der linken Seite).
A = np.zeros((N-2,N-2))
a = np.zeros((N-2))
a[:1]=Tr*(-r) #first value in vector a
a[-1:]=Tl*(-r)  #last item in vector a

B = np.zeros((N-2,N-2))
b = np.zeros((N-2))
b[:1]=Tr*r
b[-1:]=Tl*r
#define matrices A, B and b array

for i in range(N-2):
    # i ist die Zeile (Entfernung), j die Spalte der Matrizen (Zeit)
    
    if i==0:       # erste Zeile der Matrix
        A[i,:] = [2+2*r if j==0 else (-r) if j==1 else 0 for j in range(N-2)]
        B[i,:] = [2-2*r if j==0 else r if j==1 else 0 for j in range(N-2)]
#        b[i] = Tr #boundary condition at i=0 (left side of rod)
        
    elif i==N-3:     # letzte Zeile der Matrix
        A[i,:] = [-r if j==N-4 else 2+2*r if j==N-3 else 0 for j in range(N-2)]
        B[i,:] = [r if j==N-4 else 2-2*r if j==N-3 else 0 for j in range(N-2)]
#        b[i] = Tl #boundary condition at i=N
    else:
         # alle anderen Zeilen
        A[i,:] = [-r if j==i-1 or j==i+1 else 2+2*r if j==i else 0 for j in range(N-2)]
        B[i,:] = [r if j==i-1 or j==i+1 else 2-2*r if j==i else 0 for j in range(N-2)]

#initialize grid, Vector mit x für Entfernung
z = np.linspace(0,L,N)
print("A",A)
print("B",B)
print("b",b)


# initiale Bedingungen, u ist der Temperaturvektor

u = np.full(N, Ti)
u[:1] = Tr
u[-1:] = Tl

print("u", u,"z", z)
# Die Gesamtgleichung lautet A u + a = B u + b; linke Seite j+1, rechte Seite j
# bb = B u + b - a
# dann lautet die Gleichung A u = bb. Mit np.linalg.solve kann diese Gleichung nach dem Vektor u aufgelöst werden
# u ist der Vektor (array) der Temperaturen über einen Zeitschritt
# das erste und letzte Element wird u nicht zugewiesen (u[1:-1])

# evaluate right hand side at t=0

bb = B.dot(u[1:-1])+b-a
print("bb",bb)
print("z",z)
fig = plt.figure()
plt.plot(u,z,linewidth=2)
plt.xlim([Tr,Tl])

filename = 'foo000.jpg';
fig.set_tight_layout(True);
plt.xlabel("T")
plt.ylabel("z")
plt.gca().invert_yaxis()
plt.title("t = 0")
plt.savefig(filename)
plt.clf()

c = 0
for j in range(tsteps):
    print(j)
    #find solution inside domain
    u[1:-1] = np.linalg.solve(A,bb)
    print("u",u)
    #update right hand side
    bb = B.dot(u[1:-1])+b-a
    print("bb",bb)
    if(j%nplot==0): #plot results every nplot timesteps
        plt.plot(u,z,linewidth=2)
        plt.xlim([Tr,Tl])
        filename = 'foo' + str(c+1).zfill(3) + '.jpg';
        plt.xlabel("T")
        plt.ylabel("z")
        plt.gca().invert_yaxis()
        plt.title("t = %2.2f"%(dt*(j+1)))
        plt.savefig(filename)
        plt.clf()
        c += 1



A [[ 2.8478 -0.4239  0.      0.    ]
 [-0.4239  2.8478 -0.4239  0.    ]
 [ 0.     -0.4239  2.8478 -0.4239]
 [ 0.      0.     -0.4239  2.8478]]
B [[1.1522 0.4239 0.     0.    ]
 [0.4239 1.1522 0.4239 0.    ]
 [0.     0.4239 1.1522 0.4239]
 [0.     0.     0.4239 1.1522]]
b [10.5975  0.      0.     42.39  ]
u [ 25.  20.  20.  20.  20. 100.] z [0.   0.01 0.02 0.03 0.04 0.05]
bb [ 52.717  40.     40.    116.302]
z [0.   0.01 0.02 0.03 0.04 0.05]
0
u [ 25.          21.60714248  20.79693409  23.74673559  44.37398736
 100.        ]
bb [ 54.90656993  43.18773638  54.98694235 145.97394946]
1
u [ 25.          22.72990986  23.17449247  31.07660883  55.88430506
 100.        ]
bb [ 57.20806949  49.51023349  69.31949296 162.34327077]
2
u [ 25.          24.04248604  26.56315697  37.61433143  62.60551509
 100.        ]
bb [ 60.15687465  56.74239438  81.13783276 172.85878957]


<Figure size 432x288 with 0 Axes>

Wenn das Tool FFmpeg installiert ist, wird aus den erzeugten jpg-Files ein Movie erstellt. Am Ende werden die jpg-Files gelöscht. Wenn ffmpeg nicht installiert ist, können andere Programme zur Erstellung des Movies verwendet werden (z.B. QuickTime, Photoshop).

In [5]:
os.system("ffmpeg -y -i 'foo%03d.jpg' heat_equation.m4v")
os.system("rm -f *.jpg")

ffmpeg version 4.4 Copyright (c) 2000-2021 the FFmpeg developers
  built with Apple clang version 11.0.3 (clang-1103.0.32.62)
  configuration: --prefix=/opt/local --enable-swscale --enable-avfilter --enable-avresample --enable-libmp3lame --enable-libvorbis --enable-libopus --enable-librsvg --enable-libtheora --enable-libopenjpeg --enable-libmodplug --enable-libvpx --enable-libsoxr --enable-libspeex --enable-libass --enable-libbluray --enable-lzma --enable-gnutls --enable-fontconfig --enable-libfreetype --enable-libfribidi --disable-libjack --disable-libopencore-amrnb --disable-libopencore-amrwb --disable-libxcb --disable-libxcb-shm --disable-libxcb-xfixes --disable-indev=jack --enable-opencl --disable-outdev=xv --enable-audiotoolbox --enable-videotoolbox --enable-sdl2 --disable-securetransport --mandir=/opt/local/share/man --enable-shared --enable-pthreads --cc=/usr/bin/clang --enable-libdav1d --arch=x86_64 --enable-x86asm --enable-libx265 --enable-gpl --enable-postproc --enable-libx26

0

In [6]:
w=np.array([[ 2.8478, -0.4239,  0.,      0.],
 [-0.4239,  2.8478, -0.4239,  0.    ],
 [ 0.,     -0.4239,  2.8478, -0.4239],
 [ 0.,      0.,     -0.4239,  2.8478]])
print(w)

w2=np.array([116.3,40.,40.,52.718])
ww=np.linalg.solve(w,w2)
print(ww)

[[ 2.8478 -0.4239  0.      0.    ]
 [-0.4239  2.8478 -0.4239  0.    ]
 [ 0.     -0.4239  2.8478 -0.4239]
 [ 0.      0.     -0.4239  2.8478]]
[44.37327002 23.74663447 20.79697216 21.6074993 ]
